In [ ]:
import numpy as np
import json

In [ ]:
#vocabulary generate kore pura training corpus er

def generate_vocab(json_file_path, threshold=2):
    with open(json_file_path, "r" , encoding="utf-8") as json_file:
        json_data = json.load(json_file)

    vocab = {'<unk>': 0}
    counter = 0

    # each word occurs count kore
    for entry in json_data:
        sentence = entry["sentence"]
        counter+=1

        for word in sentence:
            for subword in word.split():
                if subword.strip():
                    if subword not in vocab:
                        vocab[subword] = 1
                    else:
                        vocab[subword] += 1

    # jegulo dataset e 1 bar hoese segulo count kore
    for k in list(vocab.keys())[1:]:
        if vocab[k] < threshold:
            vocab['<unk>'] += vocab[k]
            del vocab[k]
    #vocab['<unk>'] = 0
    # descending order e vocab sort kore
    sorted_vocab = {k: v for k, v in sorted(vocab.items(), key=lambda item: item[1], reverse=True)}

    # <unk> k top e push kore
    unk_val = sorted_vocab.pop('<unk>')
    sorted_vocab = {'<unk>': unk_val, **sorted_vocab}

    with open('vocab.txt', 'w', encoding="utf-8") as f:
        format_str = ''
        for index, (key, count) in enumerate(sorted_vocab.items()):
            format_str += key + '\t' + str(index) + '\t' + str(count) + '\n'
        f.write(format_str)

    #print("Threshold =", threshold)

    return sorted_vocab, counter

vocab, counter = generate_vocab("C:/Users/Admin/Desktop/Thesis/POS Tagging/archive/train_bangla.json")
print("Overall size of my vocabulary =", len(vocab))
#print("No. of times '<unk>' occurs in my vocabulary =", vocab['<unk>'])
print("No. of sentence =", counter)

In [ ]:
def load_training_data(file_path):
    with open(file_path, 'r', encoding="utf-8") as json_file:
        return json.load(json_file)

#transition holo NNP to VBP kotobar, emission holo "Book" word ta kotobar NNP r kotobar VBP
#unique state holo distinct tag gulo niye banano list
def process_training_data(data, vocab):
    transition_probabilities = {}
    emission_probabilities = {}
    unique_states = []

    for entry in data:
        sentence_tokens = entry['sentence']
        labels = entry['labels']
        prior_state = 'root'  

        for word, tag in zip(sentence_tokens, labels):
            if word not in vocab.keys():
                word = '<unk>'

            # Update unique states
            if tag not in unique_states:
                unique_states.append(tag)

            # Update transition count
            transition_key = (prior_state, tag)
            if transition_key not in transition_probabilities:
                transition_probabilities[transition_key] = 1
            else:
                transition_probabilities[transition_key] += 1

            # Update emission count
            emission_key = (tag, word)
            if emission_key not in emission_probabilities:
                emission_probabilities[emission_key] = 1
            else:
                emission_probabilities[emission_key] += 1

            prior_state = tag

    return transition_probabilities, emission_probabilities, unique_states

# normalize kore
def normalize_probabilities(probabilities):
    normalized_probabilities = {}
    for key in probabilities:
        state, value = key
        total = sum(v for k, v in probabilities.items() if k[0] == state)
        normalized_probabilities[key] = probabilities[key] / total
    return normalized_probabilities

#transition r emission gulo k dictionary er moto kore dekhano hoyese. hmm.json e ase sobgulo
def save_hmm_model(transition_probs, emission_probs, output_file):
    
    # Convert tuple keys to strings
    transition_probs = {str(k): v for k, v in transition_probs.items()}
    emission_probs = {str(k): v for k, v in emission_probs.items()}

    hmm_model = {
        'transition': transition_probs,
        'emission': emission_probs,
    }

    with open(output_file, 'w') as f:
        json.dump(hmm_model, f)

    return transition_probs, emission_probs

training_data = load_training_data('C:/Users/Admin/Desktop/Thesis/POS Tagging/archive/train_bangla.json')
transition_probs, emission_probs, unique_states = process_training_data(training_data, vocab)
normalized_transition_probs = normalize_probabilities(transition_probs)
normalized_emission_probs = normalize_probabilities(emission_probs)
transition_param_dict, emission_param_dict = save_hmm_model(normalized_transition_probs, normalized_emission_probs, 'hmm.json')

print("No. of transition parameters =", len(transition_param_dict))
print("No. of emission parameters =", len(emission_param_dict))


In [ ]:
import nltk

In [ ]:
# formatted_data er transition r emission dey 
def greedy_decoding(formatted_data, vocab, transition_param_dict, emission_param_dict, state_track):
    #accuracy = []
    transition_prob ={}
    emission_prob = {}
    #max_stt = {}

    prior_st = 'root'
    counter = 0
    for i in formatted_data:
        print(i)
        counter = counter+1
        transition_prob[counter]=transition_param_dict.get(str((prior_st, i[1])), 1e-7)
        emission_prob[counter]=emission_param_dict.get(str((i[1], i[0])), 0)
        prior_st = i[1]

    return transition_prob, emission_prob

In [ ]:
def reverse_graph(G):
    '''reverse graph ta return kore like g[dst][src]=G[src][dst]'''
    g = {}
    for src in G.keys():
        for dst in G[src].keys():
            if dst not in g.keys():
                g[dst] = {}
            g[dst][src] = G[src][dst]
    return g


def build_max(rg, root): # rg always reverse_graph hbe
    '''max incoming edge find out kore each node er'''
    mg = {}
    for dst in rg.keys():
        if dst == root: # root thakle ignore krbe
            continue
        max_ind = -100
        max_value = -100
        for src in rg[dst].keys():
            if rg[dst][src] >= max_value:
                max_ind = src
                max_value = rg[dst][src]
        mg[dst] = {max_ind: max_value}
    return mg


def find_circle(mg):
    '''circle ase kina check kore DFS diye. na thakle None return kore'''

    for start in mg.keys():
        visited = [] # jegulo visited hoye giyese segulo ekhane thakbe
        stack = [start]
        while stack: #stack empty na hoa prjnto traverse cholbe
            n = stack.pop()
            if n in visited:# n jodi already visited hoy tahole cycle
                C = [] # empty list declare krlm
                while n not in C: # ekhane backtrack er kaj ta hoy loop diye
                    C.append(n) # n k C er moddhe rakhlm
                    n = list(mg[n].keys())[0] # n k update krbo tar parent node diye 
                return C # Cycle er node gulor list return kore
            visited.append(n)
            if n in mg.keys(): # n er kono child ase kina check kore
                stack.extend(list(mg[n].keys())) # stack er moddhe current node n er child k rakhe. etai DFS er condition
    return None # cycle na thakle None return kore


def chu_liu_edmond(G, root):
    
    # reversed graph rg[dst][src] = G[src][dst]
    rg = reverse_graph(G)
    # root er only out edge
    rg[root] = {}
    # the maximum edge select korlam for each node other than root
    mg = build_max(rg, root)

    # check if mg is a tree (contains a circle)
    C = find_circle(mg)
    # circle na thakle, mg tai max_spanning_tree
    if not C:
        return reverse_graph(mg)

    # jesob node circle kore tader k niye compact node korlm
    all_nodes = G.keys()
    vc = max(all_nodes) + 1

    # new graph holo G_prime
    #V_prime = list(set(all_nodes) - set(C)) + [vc]
    G_prime = {}
    vc_in_idx = {}
    vc_out_idx = {}
    # Now add the edges to G_prime
    for u in all_nodes:
        for v in G[u].keys():
            # incoming edge er weight calculation. subtraction hoy
            if (u not in C) and (v in C):
                if u not in G_prime.keys():
                    G_prime[u] = {}
                w = G[u][v] - list(mg[v].values())[0] 
                if (vc not in G_prime[u]) or (vc in G_prime[u] and w > G_prime[u][vc]):
                    G_prime[u][vc] = w
                    vc_in_idx[u] = v

            # outgoing edge er weight calculation. jeta max oita nibo
            elif (u in C) and (v not in C):
                if vc not in G_prime.keys():
                    G_prime[vc] = {}
                w = G[u][v]
                if (v not in G_prime[vc]) or (v in G_prime[vc] and w > G_prime[vc][v]):
                    G_prime[vc][v] = w
                    vc_out_idx[v] = u

            # Third case: if the source and dest are all not in the circle, then just add the edge to the new graph.
            elif (u not in C) and (v not in C):
                if u not in G_prime.keys():
                    G_prime[u] = {}
                G_prime[u][v] = G[u][v]

    # Recursively run the algorihtm on the new graph G_prime
    A = chu_liu_edmond(G_prime, root)
    # print(A)

    # compacted node k vangbo, erpor incoming r outgoing edge gulo identify krbo
    # always max ta choose krbo r bakigulo delete krbo
    all_nodes_A = list(A.keys())
    for src in all_nodes_A:
        # The number of out-edges varies, could be 0 or any number <=|V\C|
        if src == vc:
            for node_in in A[src].keys():
                orig_out = vc_out_idx[node_in]
                if orig_out not in A.keys():
                    A[orig_out] = {}
                A[orig_out][node_in] = G[orig_out][node_in]
        else:
            #for dst in A[src]:
            for dst in list(A[src].keys()):
                # There must be only one in-edge to vc.
                if dst == vc:
                    orig_in = vc_in_idx[src]
                    A[src][orig_in] = G[src][orig_in]
                    del A[src][dst]
    
    if vc in A:
        del A[vc]

    for node in C:
        if node != orig_in:
            src = list(mg[node].keys())[0]
            if src not in A.keys():
                A[src] = {}
            A[src][node] = mg[node][src]

    return A
